# Whole stage codegen

Fusing operators together so the generated code looks like hand optimized code:
- Identity chains of operators (“stages”)
- Compile each stage into a single function
- Functionality of a general purpose execution engine; performance as if hand built system just to run your query


In [1]:
import time

# Define a simple benchmark function for measuring time taken
def benchmark(name, f):
    startTime = time.time()
    f()
    endTime = time.time()
    print "Time taken in %s: %.4f seconds" % (name, endTime - startTime)

## How fast can Spark 1.6 sum up 1 billion numbers?

In [2]:
# This config turns off whole stage code generation, effectively changing the execution path to be similar to Spark 1.6.
spark.conf.set("spark.sql.codegen.wholeStage", "false")

In [3]:
f = lambda: spark.range(1000L * 1000 * 1000).selectExpr("sum(id)").show()
benchmark("Spark 1.6", f)

+------------------+
|           sum(id)|
+------------------+
|499999999500000000|
+------------------+

Time taken in Spark 1.6: 6.7759 seconds


## How fast can Spark 1.6 join 1 billion records?

In [4]:
# This config turns off whole stage code generation, effectively changing the execution path to be similar to Spark 1.6.
spark.conf.set("spark.sql.codegen.wholeStage", "false")

In [5]:
f = lambda: spark.range(1000L * 1000 * 1000).join(spark.range(1000L), "id").count()
benchmark("Spark 1.6", f)

Time taken in Spark 1.6: 15.4027 seconds


## How fast can Spark 2.0 sum up 1 billion numbers?

In [6]:
# Now we turn on whole stage code generation to get the full Spark 2.0 experience
spark.conf.set("spark.sql.codegen.wholeStage", "true")

In [7]:
f = lambda: spark.range(1000L * 1000 * 1000).selectExpr("sum(id)").show()
benchmark("Spark 2.0", f)

+------------------+
|           sum(id)|
+------------------+
|499999999500000000|
+------------------+

Time taken in Spark 2.0: 0.4961 seconds


## How fast can Spark 2.0 join 1 billion records?

In [8]:
spark.conf.set("spark.sql.codegen.wholeStage", "true")

In [9]:
f = lambda: spark.range(1000L * 1000 * 1000).join(spark.range(1000L), "id").count()
benchmark("Spark 2.0", f)

Time taken in Spark 2.0: 0.4610 seconds


## How fast can NumPy sum up 1 billion numbers?

In [10]:
import numpy as np

f = lambda: np.sum(np.arange(1000L * 1000 * 1000))
benchmark("NumPy", f)

Time taken in NumPy: 17.0047 seconds


## How fast can Pandas join 100 million numbers?

In [11]:
import numpy as np
import pandas as pd

a = pd.DataFrame({'id': np.arange(1000L * 1000 * 100)})
b = pd.DataFrame({'id': np.arange(1000L)})
f = lambda: a.join(b, on='id', how='inner', lsuffix="_left", rsuffix="_right")
benchmark("Pandas", f)

Time taken in Pandas: 27.2142 seconds


Note: for this example, there's an obviously faster way if we can assume that we can just use indexing. For arbitrary keys, however, this approach won't work.

In [12]:
X = np.arange(1000L * 1000 * 1000)
keys = np.arange(1000L)
f = lambda: np.shape(X[keys])[0]
benchmark("Numpy indexing", f)

Time taken in Numpy indexing: 0.0162 seconds
